<a href="https://colab.research.google.com/github/maxashoka3/Coronavirus-Tweet-Sentiment-Analysiis/blob/main/Coronavirus_Tweet_Sentiment_Analysis_Capstone_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <b><u> Project Title : Sentiment Analysis : Predicting sentiment of COVID-19 tweets</u></b>

## <b> Problem Description </b>

### This challenge asks you to build a classification model to predict the sentiment of COVID-19 tweets.The tweets have been pulled from Twitter and manual tagging has been done then.

### The names and usernames have been given codes to avoid any privacy concerns.

### You are given the following information:
1. Location
2. Tweet At
3. Original Tweet
4. Label

In [ ]:
import pandas as pd
import numpy as np
from numpy import percentile
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
import re # for regular expressions
import pandas as pd 
pd.set_option("display.max_colwidth", 200) 
import string
import nltk # for text manipulation
from nltk.stem.porter import *
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from tqdm import tqdm
from gensim.models.doc2vec import LabeledSentence
import gensim
from sklearn.linear_model import LogisticRegression
from scipy import stats 
from sklearn import metrics 
from sklearn.metrics import mean_squared_error,mean_absolute_error, make_scorer,classification_report,confusion_matrix,accuracy_score,roc_auc_score,roc_curve
from sklearn.model_selection import train_test_split,cross_val_score,KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.naive_bayes import BernoulliNB
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import xgboost as xgb
import warnings 
warnings.filterwarnings("ignore")

In [ ]:
# Reading the Coronavirus Tweet dataset
df = pd.read_csv("/content/drive/MyDrive/AlmaBetter/CoronaVirus Tweets/Coronavirus Tweets.csv", encoding='latin-1')

In [ ]:
# First look 
df.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/iFz9FAn2Pa and https://t.co/xX6ghGFzCC and https://t.co/I2NlzdxNo8,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to exchange phone numbers create contact list with phone numbers of neighbours schools employer chemist GP set up online shopping accounts if poss adequate su...,Positive
2,3801,48753,Vagabonds,16-03-2020,"Coronavirus Australia: Woolworths to give elderly, disabled dedicated shopping hours amid COVID-19 outbreak https://t.co/bInCA9Vp8P",Positive
3,3802,48754,NaN,16-03-2020,"My food stock is not the only one which is empty...\r\r\n\r\r\nPLEASE, don't panic, THERE WILL BE ENOUGH FOOD FOR EVERYONE if you do not take more than you need. \r\r\nStay calm, stay safe.\r\r\n\...",Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COVID19 outbreak.\r\r\n\r\r\nNot because I'm paranoid, but because my food stock is litteraly empty. The #coronavirus is a serious thing, but please, don...",Extremely Negative


In [ ]:
set(df['Sentiment'])

{'Extremely Negative', 'Extremely Positive', 'Negative', 'Neutral', 'Positive'}

First, will drop the **Screen Name** it is totally representated by **UserName**

In [ ]:
df.drop(['UserName','ScreenName','Location','TweetAt'], inplace = True,axis = 1)

I think **OriginalTweet** is and importtant columns so lets**Look closely** at this columns..

##### OriginalTweet

In [ ]:
df['OriginalTweet'].head(50)

In [ ]:
df['OriginalTweet'][16]

'????? ????? ????? ????? ??\r\r\n?????? ????? ??????? ????????? ?\r\r\n#????_???? ????? ???? ?????? ? #????????? ?????? ?? 13 ???? ?? ?? ???? ?????? ?? ??\r\r\n#???_???????? ????? ??? ???? ? https://t.co/51bL8P6vZh'

In [ ]:
df['OriginalTweet'][31]

"'Hole' Foods...\r\r\n\r\r\n...images from the nicest grocery store in one of the richest neighborhoods in the United States.\r\r\n\r\r\nhttps://t.co/WnQSoMtkVI\r\r\n\r\r\n#BreakingNews #Breaking #Coronavirus #CoronavirusOutbreak #COVID19 #COVID?19 #COVID_19 #COVID2019 #Collapse"

We can observe **OriginalTweet** features consist of :
* Plain Text 
* Tagged Part
* Link Part
* hasTag Part


In [ ]:
pip install neattext

     |████████████████████████████████| 114 kB 8.4 MB/s 


##### NeatText

In [ ]:
import neattext.functions as nfx

### Data Cleaning

In [ ]:
# Removing the @user
df['Tweet'] = df['OriginalTweet'].apply(lambda x: nfx.remove_userhandles(x))

In [ ]:
df[['OriginalTweet','Tweet']]

,OriginalTweet,Tweet
0,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/iFz9FAn2Pa and https://t.co/xX6ghGFzCC and https://t.co/I2NlzdxNo8,https://t.co/iFz9FAn2Pa and https://t.co/xX6ghGFzCC and https://t.co/I2NlzdxNo8
1,advice Talk to your neighbours family to exchange phone numbers create contact list with phone numbers of neighbours schools employer chemist GP set up online shopping accounts if poss adequate su...,advice Talk to your neighbours family to exchange phone numbers create contact list with phone numbers of neighbours schools employer chemist GP set up online shopping accounts if poss adequate su...
2,"Coronavirus Australia: Woolworths to give elderly, disabled dedicated shopping hours amid COVID-19 outbreak https://t.co/bInCA9Vp8P","Coronavirus Australia: Woolworths to give elderly, disabled dedicated shopping hours amid COVID-19 outbreak https://t.co/bInCA9Vp8P"
3,"My food stock is not the only one which is empty...\r\r\n\r\r\nPLEASE, don't panic, THERE WILL BE ENOUGH FOOD FOR EVERYONE if you do not take more than you need. \r\r\nStay calm, stay safe.\r\r\n\...","My food stock is not the only one which is empty...\r\r\n\r\r\nPLEASE, don't panic, THERE WILL BE ENOUGH FOOD FOR EVERYONE if you do not take more than you need. \r\r\nStay calm, stay safe.\r\r\n\..."
4,"Me, ready to go at supermarket during the #COVID19 outbreak.\r\r\n\r\r\nNot because I'm paranoid, but because my food stock is litteraly empty. The #coronavirus is a serious thing, but please, don...","Me, ready to go at supermarket during the #COVID19 outbreak.\r\r\n\r\r\nNot because I'm paranoid, but because my food stock is litteraly empty. The #coronavirus is a serious thing, but please, don..."
...,...,...
41152,Airline pilots offering to stock supermarket shelves in #NZ lockdown #COVID-19 https://t.co/cz89uA0HNp,Airline pilots offering to stock supermarket shelves in #NZ lockdown #COVID-19 https://t.co/cz89uA0HNp
41153,Response to complaint not provided citing COVID-19 related delays. Yet prompt in rejecting policy before consumer TAT is over. Way to go ?,Response to complaint not provided citing COVID-19 related delays. Yet prompt in rejecting policy before consumer TAT is over. Way to go ?
41154,"You know itÂs getting tough when @KameronWilds is rationing toilet paper #coronavirus #toiletpaper @kroger martinsville, help us out!!","You know itÂs getting tough when is rationing toilet paper #coronavirus #toiletpaper martinsville, help us out!!"
41155,Is it wrong that the smell of hand sanitizer is starting to turn me on?\r\r\n\r\r\n#coronavirus #COVID19 #coronavirus,Is it wrong that the smell of hand sanitizer is starting to turn me on?\r\r\n\r\r\n#coronavirus #COVID19 #coronavirus


In [ ]:
#Cleaning urls
df['Tweet'] = df['Tweet'].apply(nfx.remove_urls)
df[['OriginalTweet','Tweet']]

,OriginalTweet,Tweet
0,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/iFz9FAn2Pa and https://t.co/xX6ghGFzCC and https://t.co/I2NlzdxNo8,and and
1,advice Talk to your neighbours family to exchange phone numbers create contact list with phone numbers of neighbours schools employer chemist GP set up online shopping accounts if poss adequate su...,advice Talk to your neighbours family to exchange phone numbers create contact list with phone numbers of neighbours schools employer chemist GP set up online shopping accounts if poss adequate su...
2,"Coronavirus Australia: Woolworths to give elderly, disabled dedicated shopping hours amid COVID-19 outbreak https://t.co/bInCA9Vp8P","Coronavirus Australia: Woolworths to give elderly, disabled dedicated shopping hours amid COVID-19 outbreak"
3,"My food stock is not the only one which is empty...\r\r\n\r\r\nPLEASE, don't panic, THERE WILL BE ENOUGH FOOD FOR EVERYONE if you do not take more than you need. \r\r\nStay calm, stay safe.\r\r\n\...","My food stock is not the only one which is empty...\r\r\n\r\r\nPLEASE, don't panic, THERE WILL BE ENOUGH FOOD FOR EVERYONE if you do not take more than you need. \r\r\nStay calm, stay safe.\r\r\n\..."
4,"Me, ready to go at supermarket during the #COVID19 outbreak.\r\r\n\r\r\nNot because I'm paranoid, but because my food stock is litteraly empty. The #coronavirus is a serious thing, but please, don...","Me, ready to go at supermarket during the #COVID19 outbreak.\r\r\n\r\r\nNot because I'm paranoid, but because my food stock is litteraly empty. The #coronavirus is a serious thing, but please, don..."
...,...,...
41152,Airline pilots offering to stock supermarket shelves in #NZ lockdown #COVID-19 https://t.co/cz89uA0HNp,Airline pilots offering to stock supermarket shelves in #NZ lockdown #COVID-19
41153,Response to complaint not provided citing COVID-19 related delays. Yet prompt in rejecting policy before consumer TAT is over. Way to go ?,Response to complaint not provided citing COVID-19 related delays. Yet prompt in rejecting policy before consumer TAT is over. Way to go ?
41154,"You know itÂs getting tough when @KameronWilds is rationing toilet paper #coronavirus #toiletpaper @kroger martinsville, help us out!!","You know itÂs getting tough when is rationing toilet paper #coronavirus #toiletpaper martinsville, help us out!!"
41155,Is it wrong that the smell of hand sanitizer is starting to turn me on?\r\r\n\r\r\n#coronavirus #COVID19 #coronavirus,Is it wrong that the smell of hand sanitizer is starting to turn me on?\r\r\n\r\r\n#coronavirus #COVID19 #coronavirus


In [ ]:
# Removing Hashtags
df['Tweet'] = df['Tweet'].apply(nfx.remove_hashtags)

In [ ]:
df[['OriginalTweet','Tweet']]

,OriginalTweet,Tweet
0,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/iFz9FAn2Pa and https://t.co/xX6ghGFzCC and https://t.co/I2NlzdxNo8,and and
1,advice Talk to your neighbours family to exchange phone numbers create contact list with phone numbers of neighbours schools employer chemist GP set up online shopping accounts if poss adequate su...,advice Talk to your neighbours family to exchange phone numbers create contact list with phone numbers of neighbours schools employer chemist GP set up online shopping accounts if poss adequate su...
2,"Coronavirus Australia: Woolworths to give elderly, disabled dedicated shopping hours amid COVID-19 outbreak https://t.co/bInCA9Vp8P","Coronavirus Australia: Woolworths to give elderly, disabled dedicated shopping hours amid COVID-19 outbreak"
3,"My food stock is not the only one which is empty...\r\r\n\r\r\nPLEASE, don't panic, THERE WILL BE ENOUGH FOOD FOR EVERYONE if you do not take more than you need. \r\r\nStay calm, stay safe.\r\r\n\...","My food stock is not the only one which is empty...\r\r\n\r\r\nPLEASE, don't panic, THERE WILL BE ENOUGH FOOD FOR EVERYONE if you do not take more than you need. \r\r\nStay calm, stay safe.\r\r\n\..."
4,"Me, ready to go at supermarket during the #COVID19 outbreak.\r\r\n\r\r\nNot because I'm paranoid, but because my food stock is litteraly empty. The #coronavirus is a serious thing, but please, don...","Me, ready to go at supermarket during the outbreak.\r\r\n\r\r\nNot because I'm paranoid, but because my food stock is litteraly empty. The is a serious thing, but please, don't panic. It cause..."
...,...,...
41152,Airline pilots offering to stock supermarket shelves in #NZ lockdown #COVID-19 https://t.co/cz89uA0HNp,Airline pilots offering to stock supermarket shelves in lockdown
41153,Response to complaint not provided citing COVID-19 related delays. Yet prompt in rejecting policy before consumer TAT is over. Way to go ?,Response to complaint not provided citing COVID-19 related delays. Yet prompt in rejecting policy before consumer TAT is over. Way to go ?
41154,"You know itÂs getting tough when @KameronWilds is rationing toilet paper #coronavirus #toiletpaper @kroger martinsville, help us out!!","You know itÂs getting tough when is rationing toilet paper martinsville, help us out!!"
41155,Is it wrong that the smell of hand sanitizer is starting to turn me on?\r\r\n\r\r\n#coronavirus #COVID19 #coronavirus,Is it wrong that the smell of hand sanitizer is starting to turn me on?\r\r\n\r\r\n


In [ ]:
# remove special characters, numbers, punctuations
df['Tweet'] = df['Tweet'].str.replace('[^a-zA-Z#]+',' ')

In [ ]:
df[['OriginalTweet','Tweet']]

,OriginalTweet,Tweet
0,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/iFz9FAn2Pa and https://t.co/xX6ghGFzCC and https://t.co/I2NlzdxNo8,and and
1,advice Talk to your neighbours family to exchange phone numbers create contact list with phone numbers of neighbours schools employer chemist GP set up online shopping accounts if poss adequate su...,advice Talk to your neighbours family to exchange phone numbers create contact list with phone numbers of neighbours schools employer chemist GP set up online shopping accounts if poss adequate su...
2,"Coronavirus Australia: Woolworths to give elderly, disabled dedicated shopping hours amid COVID-19 outbreak https://t.co/bInCA9Vp8P",Coronavirus Australia Woolworths to give elderly disabled dedicated shopping hours amid COVID outbreak
3,"My food stock is not the only one which is empty...\r\r\n\r\r\nPLEASE, don't panic, THERE WILL BE ENOUGH FOOD FOR EVERYONE if you do not take more than you need. \r\r\nStay calm, stay safe.\r\r\n\...",My food stock is not the only one which is empty PLEASE don t panic THERE WILL BE ENOUGH FOOD FOR EVERYONE if you do not take more than you need Stay calm stay safe
4,"Me, ready to go at supermarket during the #COVID19 outbreak.\r\r\n\r\r\nNot because I'm paranoid, but because my food stock is litteraly empty. The #coronavirus is a serious thing, but please, don...",Me ready to go at supermarket during the outbreak Not because I m paranoid but because my food stock is litteraly empty The is a serious thing but please don t panic It causes shortage
...,...,...
41152,Airline pilots offering to stock supermarket shelves in #NZ lockdown #COVID-19 https://t.co/cz89uA0HNp,Airline pilots offering to stock supermarket shelves in lockdown
41153,Response to complaint not provided citing COVID-19 related delays. Yet prompt in rejecting policy before consumer TAT is over. Way to go ?,Response to complaint not provided citing COVID related delays Yet prompt in rejecting policy before consumer TAT is over Way to go
41154,"You know itÂs getting tough when @KameronWilds is rationing toilet paper #coronavirus #toiletpaper @kroger martinsville, help us out!!",You know it s getting tough when is rationing toilet paper martinsville help us out
41155,Is it wrong that the smell of hand sanitizer is starting to turn me on?\r\r\n\r\r\n#coronavirus #COVID19 #coronavirus,Is it wrong that the smell of hand sanitizer is starting to turn me on


In [ ]:
# remove short words
df['Tweet'] = df['Tweet'].apply(lambda x: ' '.join([w for w in x.split() if len(w) > 2]))


###### Removing the Multiple WhiteSpaces

In [ ]:
df['Tweet'] = df['Tweet'].apply(nfx.remove_multiple_spaces)
df[['OriginalTweet','Tweet']]

,OriginalTweet,Tweet
0,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/iFz9FAn2Pa and https://t.co/xX6ghGFzCC and https://t.co/I2NlzdxNo8,and and
1,advice Talk to your neighbours family to exchange phone numbers create contact list with phone numbers of neighbours schools employer chemist GP set up online shopping accounts if poss adequate su...,advice Talk your neighbours family exchange phone numbers create contact list with phone numbers neighbours schools employer chemist set online shopping accounts poss adequate supplies regular med...
2,"Coronavirus Australia: Woolworths to give elderly, disabled dedicated shopping hours amid COVID-19 outbreak https://t.co/bInCA9Vp8P",Coronavirus Australia Woolworths give elderly disabled dedicated shopping hours amid COVID outbreak
3,"My food stock is not the only one which is empty...\r\r\n\r\r\nPLEASE, don't panic, THERE WILL BE ENOUGH FOOD FOR EVERYONE if you do not take more than you need. \r\r\nStay calm, stay safe.\r\r\n\...",food stock not the only one which empty PLEASE don panic THERE WILL ENOUGH FOOD FOR EVERYONE you not take more than you need Stay calm stay safe
4,"Me, ready to go at supermarket during the #COVID19 outbreak.\r\r\n\r\r\nNot because I'm paranoid, but because my food stock is litteraly empty. The #coronavirus is a serious thing, but please, don...",ready supermarket during the outbreak Not because paranoid but because food stock litteraly empty The serious thing but please don panic causes shortage
...,...,...
41152,Airline pilots offering to stock supermarket shelves in #NZ lockdown #COVID-19 https://t.co/cz89uA0HNp,Airline pilots offering stock supermarket shelves lockdown
41153,Response to complaint not provided citing COVID-19 related delays. Yet prompt in rejecting policy before consumer TAT is over. Way to go ?,Response complaint not provided citing COVID related delays Yet prompt rejecting policy before consumer TAT over Way
41154,"You know itÂs getting tough when @KameronWilds is rationing toilet paper #coronavirus #toiletpaper @kroger martinsville, help us out!!",You know getting tough when rationing toilet paper martinsville help out
41155,Is it wrong that the smell of hand sanitizer is starting to turn me on?\r\r\n\r\r\n#coronavirus #COVID19 #coronavirus,wrong that the smell hand sanitizer starting turn


In [ ]:
df1= df.copy()

###### Removing the Stop word

In [ ]:
''dir(nfx)

In [ ]:
#remove_stopwords
# Remove Stopwords
df['Tweet'] = df['Tweet'].apply(nfx.remove_stopwords)
df[['OriginalTweet','Tweet']]

,OriginalTweet,Tweet
0,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/iFz9FAn2Pa and https://t.co/xX6ghGFzCC and https://t.co/I2NlzdxNo8,
1,advice Talk to your neighbours family to exchange phone numbers create contact list with phone numbers of neighbours schools employer chemist GP set up online shopping accounts if poss adequate su...,advice Talk neighbours family exchange phone numbers create contact list phone numbers neighbours schools employer chemist set online shopping accounts poss adequate supplies regular meds order
2,"Coronavirus Australia: Woolworths to give elderly, disabled dedicated shopping hours amid COVID-19 outbreak https://t.co/bInCA9Vp8P",Coronavirus Australia Woolworths elderly disabled dedicated shopping hours amid COVID outbreak
3,"My food stock is not the only one which is empty...\r\r\n\r\r\nPLEASE, don't panic, THERE WILL BE ENOUGH FOOD FOR EVERYONE if you do not take more than you need. \r\r\nStay calm, stay safe.\r\r\n\...",food stock panic FOOD need Stay calm stay safe
4,"Me, ready to go at supermarket during the #COVID19 outbreak.\r\r\n\r\r\nNot because I'm paranoid, but because my food stock is litteraly empty. The #coronavirus is a serious thing, but please, don...",ready supermarket outbreak paranoid food stock litteraly thing panic causes shortage
...,...,...
41152,Airline pilots offering to stock supermarket shelves in #NZ lockdown #COVID-19 https://t.co/cz89uA0HNp,Airline pilots offering stock supermarket shelves lockdown
41153,Response to complaint not provided citing COVID-19 related delays. Yet prompt in rejecting policy before consumer TAT is over. Way to go ?,Response complaint provided citing COVID related delays prompt rejecting policy consumer TAT Way
41154,"You know itÂs getting tough when @KameronWilds is rationing toilet paper #coronavirus #toiletpaper @kroger martinsville, help us out!!",know getting tough rationing toilet paper martinsville help
41155,Is it wrong that the smell of hand sanitizer is starting to turn me on?\r\r\n\r\r\n#coronavirus #COVID19 #coronavirus,wrong smell hand sanitizer starting turn


In [ ]:
df.head()

,OriginalTweet,Sentiment,Tweet
0,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/iFz9FAn2Pa and https://t.co/xX6ghGFzCC and https://t.co/I2NlzdxNo8,Neutral,
1,advice Talk to your neighbours family to exchange phone numbers create contact list with phone numbers of neighbours schools employer chemist GP set up online shopping accounts if poss adequate su...,Positive,advice Talk neighbours family exchange phone numbers create contact list phone numbers neighbours schools employer chemist set online shopping accounts poss adequate supplies regular meds order
2,"Coronavirus Australia: Woolworths to give elderly, disabled dedicated shopping hours amid COVID-19 outbreak https://t.co/bInCA9Vp8P",Positive,Coronavirus Australia Woolworths elderly disabled dedicated shopping hours amid COVID outbreak
3,"My food stock is not the only one which is empty...\r\r\n\r\r\nPLEASE, don't panic, THERE WILL BE ENOUGH FOOD FOR EVERYONE if you do not take more than you need. \r\r\nStay calm, stay safe.\r\r\n\...",Positive,food stock panic FOOD need Stay calm stay safe
4,"Me, ready to go at supermarket during the #COVID19 outbreak.\r\r\n\r\r\nNot because I'm paranoid, but because my food stock is litteraly empty. The #coronavirus is a serious thing, but please, don...",Extremely Negative,ready supermarket outbreak paranoid food stock litteraly thing panic causes shortage


In [ ]:
df['clean_tweet'][41156]

'new/used Rift going $700.00 Amazon rn normal market price usually $400.00 . Prices crazy right vr headsets HL Alex announced worse COVID-19. whethe'

The tweet has still **backslash left(\)** so we will remove it

In [ ]:
df['clean_tweet'][41156]

'new used Rift going Amazon rn normal market price usually Prices crazy right vr headsets HL Alex announced worse COVID whethe'

In [ ]:
df['clean_tweet'][4]

In [ ]:
df2=df.copy()

In [ ]:
df2[df2['Tweet'] == ''].shape,df2.shape

((83, 3), (41157, 3))

In [ ]:
# Selecting non- blank
df2 = df2[df2['Tweet'] != '']

In [ ]:
df2[df2['Tweet'] == ''].shape,df2.shape

((0, 3), (41074, 3))

In [ ]:
df2.shape,df.shape

((41074, 3), (41157, 3))

In [ ]:
df2.head()

,OriginalTweet,Sentiment,Tweet
1,advice Talk to your neighbours family to exchange phone numbers create contact list with phone numbers of neighbours schools employer chemist GP set up online shopping accounts if poss adequate su...,Positive,advice Talk neighbours family exchange phone numbers create contact list phone numbers neighbours schools employer chemist set online shopping accounts poss adequate supplies regular meds order
2,"Coronavirus Australia: Woolworths to give elderly, disabled dedicated shopping hours amid COVID-19 outbreak https://t.co/bInCA9Vp8P",Positive,Coronavirus Australia Woolworths elderly disabled dedicated shopping hours amid COVID outbreak
3,"My food stock is not the only one which is empty...\r\r\n\r\r\nPLEASE, don't panic, THERE WILL BE ENOUGH FOOD FOR EVERYONE if you do not take more than you need. \r\r\nStay calm, stay safe.\r\r\n\...",Positive,food stock panic FOOD need Stay calm stay safe
4,"Me, ready to go at supermarket during the #COVID19 outbreak.\r\r\n\r\r\nNot because I'm paranoid, but because my food stock is litteraly empty. The #coronavirus is a serious thing, but please, don...",Extremely Negative,ready supermarket outbreak paranoid food stock litteraly thing panic causes shortage
5,"As news of the regionÂs first confirmed COVID-19 case came out of Sullivan County last week, people flocked to area stores to purchase cleaning supplies, hand sanitizer, food, toilet paper and ot...",Positive,news region confirmed COVID case came Sullivan County week people flocked area stores purchase cleaning supplies hand sanitizer food toilet paper goods reports


### Features Extraction

In [ ]:
new_df = df[['Tweet','Sentiment']]

In [ ]:
# tokenizing tweet into new variables
tokenized_tweet = new_df['Tweet'].apply(lambda x: x.split())


In [ ]:
len(tokenized_tweet)

41157

In [ ]:
nltk.download('wordnet')
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
 
lemmatizer = WordNetLemmatizer()

# apply stemmer for tokenized_tweet
tokenized_tweet = tokenized_tweet.apply(lambda x: [lemmatizer.lemmatize(i) for i in x])
'''
from nltk.stem.porter import *
stemmer = PorterStemmer()

# apply stemmer for tokenized_tweet
tokenized_tweet = tokenized_tweet.apply(lambda x: [stemmer.stem(i) for i in x])
'''


'\nfrom nltk.stem.porter import *\nstemmer = PorterStemmer()\n\n# apply stemmer for tokenized_tweet\ntokenized_tweet = tokenized_tweet.apply(lambda x: [stemmer.stem(i) for i in x])\n'

In [ ]:
len(tokenized_tweet)

41074

In [ ]:
# join tokens into one sentence
for i in range(len(tokenized_tweet)):
    tokenized_tweet[i] = ' '.join(tokenized_tweet[i])
# change df['Tweet'] to tokenized_tweet

In [ ]:
new_df['Tweet']  = tokenized_tweet

In [ ]:
new_df.head()


,Tweet,Sentiment
0,,Neutral
1,advice Talk neighbour family exchange phone number create contact list phone number neighbour school employer chemist set online shopping account po adequate supply regular med order,Positive
2,Coronavirus Australia Woolworths elderly disabled dedicated shopping hour amid COVID outbreak,Positive
3,food stock panic FOOD need Stay calm stay safe,Positive
4,ready supermarket outbreak paranoid food stock litteraly thing panic cause shortage,Extremely Negative


Checking blank Tweet which may be created While Data Cleaning process

In [ ]:
new_df= new_df[new_df['Tweet'] != '']

In [ ]:
new_df = new_df1.copy()

(41157, 2)

In [ ]:
new_df.head()

,Tweet,Sentiment
1,advice Talk neighbour family exchange phone number create contact list phone number neighbour school employer chemist set online shopping account po adequate supply regular med order,Positive
2,Coronavirus Australia Woolworths elderly disabled dedicated shopping hour amid COVID outbreak,Positive
3,food stock panic FOOD need Stay calm stay safe,Positive
4,ready supermarket outbreak paranoid food stock litteraly thing panic cause shortage,Extremely Negative
5,news region confirmed COVID case came Sullivan County week people flocked area store purchase cleaning supply hand sanitizer food toilet paper good report,Positive


In [ ]:
df['Sentiment'].value_counts()

Positive              11422
Negative               9917
Neutral                7713
Extremely Positive     6624
Extremely Negative     5481
Name: Sentiment, dtype: int64

In [ ]:
new_df.head()

,Tweet,Sentiment
1,advice Talk neighbour family exchange phone number create contact list phone number neighbour school employer chemist set online shopping account po adequate supply regular med order,Positive
2,Coronavirus Australia Woolworths elderly disabled dedicated shopping hour amid COVID outbreak,Positive
3,food stock panic FOOD need Stay calm stay safe,Positive
4,ready supermarket outbreak paranoid food stock litteraly thing panic cause shortage,Extremely Negative
5,news region confirmed COVID case came Sullivan County week people flocked area store purchase cleaning supply hand sanitizer food toilet paper good report,Positive


In [ ]:
new_df.isnull().sum()

Tweet        0
Sentiment    0
dtype: int64

In [ ]:
new_df.head(7)

,clean_tweet,Sentiment
1,advice Talk neighbours family exchange phone n...,Positive
2,Coronavirus Australia Woolworths elderly disab...,Positive
3,food stock empty PLEASE panic FOOD need Stay c...,Positive
4,ready supermarket outbreak paranoid food stock...,Extremely Negative
5,news region confirmed COVID case came Sullivan...,Positive
6,Cashier grocery store sharing insights prove c...,Positive
7,supermarket today buy toilet paper,Neutral


In [ ]:
new_df.head()

,Tweet,Sentiment
0,and and,Neutral
1,advic talk your neighbour famili exchang phone number creat contact list with phone number neighbour school employ chemist set onlin shop account poss adequ suppli regular med but not over order,Positive
2,coronaviru australia woolworth give elderli disabl dedic shop hour amid covid outbreak,Positive
3,food stock not the onli one which empti pleas don panic there will enough food for everyon you not take more than you need stay calm stay safe,Positive
4,readi supermarket dure the outbreak not becaus paranoid but becaus food stock litterali empti the seriou thing but pleas don panic caus shortag,Extremely Negative


### Machine Learning Model

In [ ]:
from sklearn.model_selection import train_test_split

train,valid = train_test_split(new_df,test_size = 0.2,random_state=0,stratify = new_df.Sentiment.values) #stratification means that the train_test_split method returns training and test subsets that have the same proportions of class labels as the input dataset.
print("train shape : ", train.shape)
print("valid shape : ", valid.shape)

train shape :  (32859, 2)
valid shape :  (8215, 2)


In [ ]:
###Using of Counter Vectorizer for MultiClass Classification

from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
stop = list(stopwords.words('english'))
vectorizer = CountVectorizer(decode_error = 'replace',)

X_train = vectorizer.fit_transform(train.Tweet.values)
X_valid = vectorizer.transform(valid.Tweet.values)

y_train = train.Sentiment.values
y_valid = valid.Sentiment.values

print("X_train.shape : ", X_train.shape)
print("X_train.shape : ", X_valid.shape)
print("y_train.shape : ", y_train.shape)
print("y_valid.shape : ", y_valid.shape)

X_train.shape :  (32859, 26848)
X_train.shape :  (8215, 26848)
y_train.shape :  (32859,)
y_valid.shape :  (8215,)


In [ ]:
from sklearn.linear_model import SGDClassifier

sgd_clf = SGDClassifier(loss = 'hinge', penalty = 'l2', random_state=0)

sgd_clf.fit(X_train,y_train)

sgd_prediction = sgd_clf.predict(X_valid)
sgd_accuracy = accuracy_score(y_valid,sgd_prediction)
print("Training accuracy Score    : ",sgd_clf.score(X_train,y_train))
print("Validation accuracy Score : ",sgd_accuracy )
print(classification_report(sgd_prediction,y_valid))

Training accuracy Score    :  0.8553516540369457
Validation accuracy Score :  0.582227632379793
                    precision    recall  f1-score   support

Extremely Negative       0.70      0.62      0.66      1232
Extremely Positive       0.67      0.64      0.65      1402
          Negative       0.41      0.54      0.47      1519
           Neutral       0.79      0.60      0.68      2037
          Positive       0.48      0.54      0.51      2025

          accuracy                           0.58      8215
         macro avg       0.61      0.59      0.59      8215
      weighted avg       0.61      0.58      0.59      8215



In [ ]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()

logreg.fit(X_train, y_train)

logreg_prediction = logreg.predict(X_valid)
logreg_accuracy = accuracy_score(y_valid,logreg_prediction)
print("Training accuracy Score    : ",logreg.score(X_train,y_train))
print("Validation accuracy Score : ",logreg_accuracy )
print(classification_report(logreg_prediction,y_valid))

Training accuracy Score    :  0.9168264402446819
Validation accuracy Score :  0.6275106512477175
                    precision    recall  f1-score   support

Extremely Negative       0.60      0.68      0.64       968
Extremely Positive       0.61      0.70      0.65      1144
          Negative       0.57      0.57      0.57      1980
           Neutral       0.76      0.65      0.70      1763
          Positive       0.61      0.59      0.60      2360

          accuracy                           0.63      8215
         macro avg       0.63      0.64      0.63      8215
      weighted avg       0.63      0.63      0.63      8215



In [ ]:
from sklearn.naive_bayes import MultinomialNB

naiveByes_clf = MultinomialNB()

naiveByes_clf.fit(X_train,y_train)

NB_prediction = naiveByes_clf.predict(X_valid)
NB_accuracy = accuracy_score(y_valid,NB_prediction)
print("training accuracy Score    : ",naiveByes_clf.score(X_train,y_train))
print("Validation accuracy Score : ",NB_accuracy )
print(classification_report(NB_prediction,y_valid))

training accuracy Score    :  0.7180985422563072
Validation accuracy Score :  0.48204503956177724
                    precision    recall  f1-score   support

Extremely Negative       0.41      0.58      0.48       775
Extremely Positive       0.43      0.56      0.49      1016
          Negative       0.52      0.44      0.48      2331
           Neutral       0.34      0.69      0.46       751
          Positive       0.61      0.42      0.49      3342

          accuracy                           0.48      8215
         macro avg       0.46      0.54      0.48      8215
      weighted avg       0.52      0.48      0.48      8215



In [ ]:
#Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier()

rf_clf.fit(X_train,y_train)

rf_prediction = rf_clf.predict(X_valid)
rf_accuracy = accuracy_score(y_valid,rf_prediction)
print("Training accuracy Score    : ",rf_clf.score(X_train,y_train))
print("Validation accuracy Score : ",rf_accuracy )
print(classification_report(rf_prediction,y_valid))

Training accuracy Score    :  0.999604370187772
Validation accuracy Score :  0.572976262933658
                    precision    recall  f1-score   support

Extremely Negative       0.39      0.69      0.50       609
Extremely Positive       0.37      0.71      0.49       688
          Negative       0.55      0.52      0.53      2084
           Neutral       0.83      0.62      0.71      2038
          Positive       0.63      0.52      0.57      2796

          accuracy                           0.57      8215
         macro avg       0.55      0.61      0.56      8215
      weighted avg       0.62      0.57      0.58      8215



In [ ]:
#takes huge amount of time to execute
import xgboost as xgb

xgboost_clf = xgb.XGBClassifier()

xgboost_clf.fit(X_train, y_train)

xgb_prediction = xgboost_clf.predict(X_valid)
xgb_accuracy = accuracy_score(y_valid,xgb_prediction)
print("Training accuracy Score    : ",xgboost_clf.score(X_train,y_train))
print("Validation accuracy Score : ",xgb_accuracy )
print(classification_report(xgb_prediction,y_valid))

Training accuracy Score    :  0.5032715542165008
Validation accuracy Score :  0.4858186244674376
                    precision    recall  f1-score   support

Extremely Negative       0.38      0.65      0.48       650
Extremely Positive       0.41      0.68      0.51       811
          Negative       0.42      0.45      0.44      1881
           Neutral       0.60      0.47      0.53      1950
          Positive       0.55      0.43      0.49      2923

          accuracy                           0.49      8215
         macro avg       0.48      0.53      0.49      8215
      weighted avg       0.51      0.49      0.49      8215

